# Progetto 2021

In [ ]:
SEED = 9246

import os
import json
import time
from datetime import datetime
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import models

from PIL import Image

from skimage import io
import pandas as pd
import numpy as np
np.random.seed(SEED)

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import classification_report

import torchsummary

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

if CUDA:
    print('run on cuda %s' % os.environ['CUDA_VISIBLE_DEVICES'])

In [ ]:
DATASET_DIR = 'progetto_2021_dataset'
DRIVE_FOLDER = 'progetto_2021/model_checkpoint'

JSON_DATA = os.path.join(DATASET_DIR, 'train_test_split_dict.json')

with open(JSON_DATA) as fp:
    dataset_json = json.load(fp)
    
len(dataset_json)

In [ ]:
labels = set()

for k in dataset_json.values():
    for lable_list in k.values():
        for v in lable_list:
            labels.add(v)
            
label_idx = {v: i for i, v in enumerate(sorted(labels))}

# Load data

In [ ]:
class VideoDataset(Dataset):

    def __init__(self, dataset_folder, labels_dict, transform=None, limit=30):
        """
        Args:
            dataset_folder (string): Path to the folder with mp4 files.
            labels_dict (dict): dict filename - list of label.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels_dict = labels_dict
        self.root_dir = dataset_folder
        self.limit = limit
        self.transform = transform or torchvision.transforms.ToTensor()
        
        self._files = np.array(list(self.labels_dict.keys()))

    def __len__(self):
        return len(self.labels_dict)

    def __getitem__(self, idx):
        name = self._files[idx]
        
        x = torch.zeros(self.limit, 3, 224, 224)
        folder_pattern = os.path.join(self.root_dir, name, '*.png')
        images = io.imread_collection(folder_pattern)
        
        for i, image in enumerate(images):
            if i < self.limit:
                image = self.transform(Image.fromarray(image))
                x[i] = image.unsqueeze(0)

        labels = torch.zeros(len(label_idx), dtype=torch.float32)
        for label in self.labels_dict[name]:
            labels[label_idx[label]] = 1
        
        return x, labels


transformations = torchvision.transforms.Compose([torchvision.transforms.CenterCrop(224),
                                                  torchvision.transforms.ToTensor()])

datasetTrain = VideoDataset(DATASET_DIR, dataset_json['train'], transformations)
datasetTest = VideoDataset(DATASET_DIR, dataset_json['test'], transformations)

# Dataset

In [ ]:
batch_size = 16
trainingDataLoader = torch.utils.data.DataLoader(datasetTrain, 
                                                 batch_size=batch_size, 
                                                 shuffle=True, 
                                                 num_workers=4)
testDataLoader = torch.utils.data.DataLoader(datasetTest, 
                                             batch_size=batch_size, 
                                             shuffle=True, 
                                             num_workers=4)

# Loading model

In [ ]:
CHECKPOINT = os.path.join(DRIVE_FOLDER, 'model.checkpoint')
MODELFILE = os.path.join(DRIVE_FOLDER, 'model.pth')

def save_checkpoint(epoch, model, optimizer, loss):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, CHECKPOINT)

def load_checkpoint(model, optimizer):
    if not os.path.exists(CHECKPOINT):
        return None, None
    checkpoint = torch.load(CHECKPOINT)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    return epoch, loss

def save_model(model):
    torch.save(model.state_dict(), MODELFILE)

def load_lodel(model):
    if os.path.exists(MODELFILE):
        model.load_state_dict(torch.load(MODELFILE))


In [ ]:
epochs = 50
train_losses = []

print_every = 100

try:
    for epoch in range(last_epoch or 0, epochs):
        print('Start epoch', epoch+1)
        model.train()
        running_loss = 0
        steps = 0
        for x, y in trainingDataLoader:

            steps += 1
            x, y = x.to(device), y.to(device)

            if epoch == 0 and steps == 1:
                print(f'input shape is {x.shape}, labels are {y.shape}')

            optimizer.zero_grad()
            logit = model(x)

            loss = criterion(logit, y)
            loss.backward()

            optimizer.step()
            running_loss += loss.item()
            
            # TRAIN unlabeled batch
            # ...
            
            if steps % print_every == 0:
                print(f"epoch {epoch+1}/{epochs} "
                      f"train loss: {running_loss/steps:.6f} ")

                save_checkpoint(epoch, model, optimizer, loss)

        save_model(model)
        train_losses.append(running_loss/steps)
        print(f"END Epoch {epoch+1}/{epochs} "
              f"Train loss: {running_loss/steps:.6f} ")
        
        if (epoch + 1) % 10 == 0:
            print('*'*50)
            test(model)
            print('*'*50)
except KeyboardInterrupt: 
    print('Exiting from training early')

save_model(model)

# Plot loss

# Test

In [ ]:
topk=10
predictions = []
y_true = []

model.eval()

with torch.no_grad():
    for inputs, labels in testDataLoader:
        inputs = inputs.to(device)

        logps = model(inputs)
        y_pred = torch.sigmoid(logps)

        _, idx = y_pred.topk(topk, dim=1)

        y_pred = torch.zeros_like(y_pred)
        y_pred.scatter_(1, idx, 1)
        predictions.append(y_pred.cpu())

        y_true.append(labels.cpu())


y_true, predictions = torch.cat(y_true, axis=0), torch.cat(predictions, axis=0)
report = classification_report(y_true, predictions, 
                               target_names=list(sorted(label_idx.keys())))
print(report)